In [72]:
import logging
import os
import pickle
import time
import statistics
from abc import abstractmethod

import colorsys
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import hdbscan

from sklearn.cluster import OPTICS, cluster_optics_dbscan
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from sklearn.metrics import silhouette_score, silhouette_samples
import statistics



<font size = 5>Load Data, Create airspeedchange feature and Drop other rows</font>

In [73]:
f0 = 'base.csv'
f1 = 'random1.csv'
f2 = 'wind1.csv'
f3 = 'engine1.csv'
f4 = 'sensor1.csv'
f5 = 'crash.csv'

base_df = pd.read_csv(f0, header=0, index_col=None)
random1_df = pd.read_csv(f1, header=0, index_col=None)
wind1_df = pd.read_csv(f2, header=0, index_col=None)
engine1_df = pd.read_csv(f3, header=0, index_col=None)
sensor1_df = pd.read_csv(f4, header=0, index_col=None)
crash1_df = pd.read_csv(f5, header=0, index_col=None)

df_list = [base_df, random1_df, wind1_df, engine1_df, sensor1_df,crash1_df]

for df in df_list:

    df['airspeedchange'] = df['airspeed']- df['airspeed'].shift(1)
    df['airspeedchange'].fillna(0, inplace=True)

base_df = base_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed',  'airspeedchange']]
random1_df = random1_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed', 'airspeedchange']]
wind1_df = wind1_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed', 'airspeedchange']]
engine1_df = engine1_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed', 'airspeedchange']]
sensor1_df = sensor1_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed', 'airspeedchange']]
crash1_df = crash1_df[['roll', 'pitch', 'yaw', 'rollspeed', 'pitchspeed', 'yawspeed', 'airspeedchange']]



<font size = 5>Count anomalies</font>

In [74]:
random1_nonanomalies = len(random1_df)
wind1_anomalies = len(wind1_df)
engine1_anomalies = len(engine1_df)
sensor1_anomalies = len(sensor1_df)
crash1_anomalies = len(crash1_df)


<font size = 5>Train SVM Model with Base Dataset</font>

In [75]:
from sklearn.svm import OneClassSVM

nu = 0.1
kernel = 'rbf'
gamma = 'scale'

# Define and train the model
model = OneClassSVM(nu=nu, kernel=kernel, gamma=gamma)
model.fit(base_df)

OneClassSVM(nu=0.1)

<font size = 5>Predicting using random missions (RQ1)</font>

In [76]:
pred = model.predict(random1_df)

<font size = 5> Finding number of nonanomalies for random missions </font>

In [77]:
count = 0

for value in pred:
    if value == -1:
        count += 1

nonanomalies = random1_nonanomalies - count
nonanomaly_percentage = nonanomalies/random1_nonanomalies*100

print(nonanomalies, "of the observations are nonanomalies, which is equal to", nonanomaly_percentage,"%")

331 of the observations are nonanomalies, which is equal to 93.50282485875707 %


<font size = 5> Predicting using Wind Proxy (RQ2 1/3) </font>

In [78]:
pred = model.predict(wind1_df)

<font size = 5> Finding number of outliers for Wind proxy </font>

In [79]:
count = 0

for value in pred:
    if value == -1:
        count += 1
        
anomaly_percentage = count/wind1_anomalies*100

print(count, "of the observations are anomalies, which is equal to", anomaly_percentage,"%")

569 of the observations are anomalies, which is equal to 84.42136498516321 %


<font size = 5> Predicting using Engine Faults (RQ2 2/3) </font>



In [80]:
pred = model.predict(engine1_df)

<font size = 5> Finding number of outliers for Engine Faults </font>

In [81]:
count = 0

for value in pred:
    if value == -1:
        count += 1
        
anomaly_percentage = count/engine1_anomalies*100

print(count, "of the observations are anomalies, which is equal to", anomaly_percentage,"%")

353 of the observations are anomalies, which is equal to 75.1063829787234 %


<font size = 5> Predicting using Manually Injected Sensor Faults (RQ2 3/3) </font>


In [82]:
pred = model.predict(sensor1_df)

<font size = 5> Finding number of outliers for Sensor Faults </font>

In [83]:
count = 0

for value in pred:
    if value == -1:
        count += 1
        
anomaly_percentage = count/sensor1_anomalies*100

print(count, "of the observations are anomalies, which is equal to", anomaly_percentage,"%")

212 of the observations are anomalies, which is equal to 100.0 %


<font size = 5> Predicting using Crash Faults (RQ2 3/3) </font>


In [84]:
pred = model.predict(crash1_df)

<font size=5>Finding number of outliers for Crash Faults </font>

In [85]:
count = 0

for value in pred:
    if value == -1:
        count += 1
        
anomaly_percentage = count/crash1_anomalies*100

print(count, "of the observations are anomalies, which is equal to", anomaly_percentage,"%")

68 of the observations are anomalies, which is equal to 91.8918918918919 %
